In [1]:
import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn import metrics

In [2]:
import os
os.chdir(r'C:\Users\Vish\Documents\Data\Dataset')
df = pd.read_csv("Loan.csv")

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.drop(['Loan_ID'],axis=1,inplace=True)
df.dropna(inplace=True)

In [5]:
#Check class imbalance
df['Loan_Status'].value_counts()

Y    332
N    148
Name: Loan_Status, dtype: int64

In [6]:
target=df['Loan_Status'] 
df.drop(['Loan_Status'],axis=1,inplace=True)
categorical_features_indices = np.where(df.dtypes == np.object)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4, 10], dtype=int64)

In [7]:
#Splitting the data into independent and dependent variables
X = df.iloc[:,:11]
y =target

In [8]:
#Evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
#Model building
model = CatBoostClassifier(n_estimators=100,learning_rate=0.01,depth=5,one_hot_max_size=3,od_pval=0.2,early_stopping_rounds=10)

#iterations:int ,default=1000--The no of trees to be built
#learning_rate:float, range=0-1--The learning rate of the model. The default value is defined automatically for Logloss, MultiClass & RMSE loss functions depending on the number of iterations if none of these parameters is set.
#In other cases, the default value is 0.03
#depth-int, range=0-Inf, default=6--The depth of the tree
#one_hot_max_size-int, range= 0-Inf, default=2--Use one-hot encoding for all categorical features with a number of different values less than or equal to the given parameter value
#l2_leaf_reg- float, range=0-Inf, default=3.0--Coefficient at the L2 regularization term of the cost function.
#bootstrap_type-{"Bayesian","Bernoulli","MVS","Poisson","No"}--Defines the method for sampling the weights of objects. 
#loss_function-{string,object}--The metric to use in training. The specified value also determines the machine learning problem to solve. 
#eval_metric-{"CrossEntropy","Logloss","RMSE","AUC","Accuracy","MultiClass","Precision","Recall","F1","R2","MultiRMSE"}--The metric used for best model selection
#nan_mode-{"Forbidden","Min","Max"}, default="Min"--The method for  processing missing values in the input dataset. "Forbidden" means Missing values are not supported and their presence is interpreted as an error
#“Min” means missing values are processed as the minimum value (less than all other values) for the feature. "Max" means missing values are processed as the maximum value (greater than all other values) for the feature
#use_best_model-bool--If this parameter is set, the number of trees that are saved in the resulting model is defined as - Build the number of trees defined by the training parameters and Use the validation dataset to identify the iteration with the optimal value of the metric specified in eval_metric.
#No trees are saved after this iteration. This option requires a validation dataset to be provided. The default value is true if a validation set is input (the eval_set parameter is defined) and at least one of the label values of objects in this set differs from the others. False otherwise.
#best_model_min_trees- int,range=0-Inf, default=None. The minimal number of trees that the best model should have. If set, the output model contains at least the given number of trees even if the best model is located within these trees. Should be used with the use_best_model parameter.
#random_strength-float, range=0-Inf, default=1--The amount of randomness to use for scoring splits when the tree structure is selected.
#bagging_temperature-float, range=0-Inf, default=1 --Defines the settings of the Bayesian bootstrap. It is used by default in classification and regression modes.
#Use the Bayesian bootstrap to assign random weights to objects. The weights are sampled from exponential distribution if the value of this parameter is set to “1”. All weights are equal to 1 if the value of this parameter is set to “0”.
#subsample-float, range=0-1--Sample rate for bagging.The default value depends on the dataset size and the bootstrap type: Datasets with less than 100 objects - 1.
#Datasets with 100 objects or more - Poisson, Bernoulli - 0.66, MVS — 0.8
#sampling_frequency-{"PerTree","PerTreeLevel"}, default=PerTree--Frequency to sample weights and objects when building trees.
#colsample_bylevel:float, range=0-1, default=1--Denotes the subsample ratio of columns for each split in each level
#scale_pos_weight:float, range=0-Inf, default=1--The weight for class 1 in binary classification. The value is used as a multiplier for the weights of objects from class 1.
#boosting_type:{"Ordered","Plain"}--Boosting scheme.
#border_count-int, range=1-65535,default=254--The number of splits for numerical features
#max_leaves-int, range=0-Inf, default=31--The maximum number of leafs in the resulting tree.
#grow_policy-{"SymmetricTree","Depthwise","Lossguide"}, default=SymmetricTree--Defines how to perform greedy tree construction.
#od_type-{"IncToDec","Iter"}, default=IncToDec--The type of the overfitting detector to use.
#od_pval-float, range=0-1, default=0--The threshold for the IncToDec overfitting detector type. The training is stopped when the specified value is reached. Requires that a validation dataset was input.
#od_wait-int, range=0-Inf, default=20--The number of iterations to continue the training after the iteration with the optimal metric value.
#early_stopping_rounds-int, range=0-Inf--Sets the overfitting detector type to Iter and stops the training after the specified number of iterations since the iteration with the optimal metric value.
#min_data_in_leaf:int , range=0-Inf, default=1--The minimum number of training samples in a leaf. CatBoost does not search for new splits in leaves with samples count less than the specified value.
#ignored_features:list, defaut=None--Feature indices or names to exclude from the training.
#random_seed:{int, None}, range=0-Inf, default=None--Can be used for generating reproducible results
#thread_count:int,default=-1--The number of threads to use during the training.
model.fit(X_train, y_train,categorical_features_indices, eval_set=(X_test ,y_test))

0:	learn: 0.6898890	test: 0.6903932	best: 0.6903932 (0)	total: 559ms	remaining: 55.4s
1:	learn: 0.6866196	test: 0.6875644	best: 0.6875644 (1)	total: 571ms	remaining: 28s
2:	learn: 0.6834285	test: 0.6847407	best: 0.6847407 (2)	total: 585ms	remaining: 18.9s
3:	learn: 0.6802647	test: 0.6818574	best: 0.6818574 (3)	total: 593ms	remaining: 14.2s
4:	learn: 0.6768162	test: 0.6787415	best: 0.6787415 (4)	total: 607ms	remaining: 11.5s
5:	learn: 0.6739652	test: 0.6765233	best: 0.6765233 (5)	total: 620ms	remaining: 9.72s
6:	learn: 0.6709265	test: 0.6738694	best: 0.6738694 (6)	total: 635ms	remaining: 8.43s
7:	learn: 0.6682337	test: 0.6716484	best: 0.6716484 (7)	total: 648ms	remaining: 7.45s
8:	learn: 0.6654236	test: 0.6693685	best: 0.6693685 (8)	total: 663ms	remaining: 6.7s
9:	learn: 0.6625937	test: 0.6671644	best: 0.6671644 (9)	total: 677ms	remaining: 6.1s
10:	learn: 0.6599215	test: 0.6651584	best: 0.6651584 (10)	total: 693ms	remaining: 5.61s
11:	learn: 0.6573152	test: 0.6628477	best: 0.6628477 (11

98:	learn: 0.5173917	test: 0.5497282	best: 0.5497282 (98)	total: 1.89s	remaining: 19.1ms
99:	learn: 0.5165072	test: 0.5490668	best: 0.5490668 (99)	total: 1.91s	remaining: 0us

bestTest = 0.5490668047
bestIteration = 99



In [10]:
#model = CatBoostRegressor()

In [11]:
#Predictions
#The predict fuction converts probability values > .5 to 1 else 0
y_pred=model.predict(data=X_test)

In [12]:
#Evaluating model
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Recall: ", metrics.precision_score(y_test,y_pred,pos_label='Y'))
print("F1 score: ", metrics.f1_score(y_test,y_pred,pos_label='Y'))

Accuracy:  0.7916666666666666
Recall:  0.76
F1 score:  0.8636363636363636


In [13]:
# Using our own threshold
# decisions = (model.predict_proba(X_test) >= 0.6).astype(int)
# y_pred=decisions[:,1]
# print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
# print("Recall: ", metrics.precision_score(y_test,y_pred))
# print("F1 score: ", metrics.f1_score(y_test,y_pred))

In [14]:
#AUC ROC
# Auc roc works on class probabilities. Although it can work on predictions of 0,1 as well but it works well with probabilities
# class probabilities are indepedent of threshold because first class probabilites are calculated and then threshold is used
#to predict 0 and 1. So auc will remain same for all threshold values
# Using probs to generat class probabilities
# probs array return two array- probability for negative class and probability for positive class
probs = model.predict_proba(X_test)
probs

array([[0.34061677, 0.65938323],
       [0.34166483, 0.65833517],
       [0.31956006, 0.68043994],
       [0.32881159, 0.67118841],
       [0.32473602, 0.67526398],
       [0.31837014, 0.68162986],
       [0.32152077, 0.67847923],
       [0.31326207, 0.68673793],
       [0.36789449, 0.63210551],
       [0.32821137, 0.67178863],
       [0.33152992, 0.66847008],
       [0.32450362, 0.67549638],
       [0.63367598, 0.36632402],
       [0.34850675, 0.65149325],
       [0.33659219, 0.66340781],
       [0.34828298, 0.65171702],
       [0.35305375, 0.64694625],
       [0.35709425, 0.64290575],
       [0.6361945 , 0.3638055 ],
       [0.28466604, 0.71533396],
       [0.32718912, 0.67281088],
       [0.3470361 , 0.6529639 ],
       [0.31016664, 0.68983336],
       [0.30663198, 0.69336802],
       [0.38789387, 0.61210613],
       [0.61101474, 0.38898526],
       [0.31249961, 0.68750039],
       [0.34127091, 0.65872909],
       [0.29665222, 0.70334778],
       [0.3251012 , 0.6748988 ],
       [0.

In [15]:
#Probability for positive class
y_pred_prob = probs[:, 1]
y_pred_prob

array([0.65938323, 0.65833517, 0.68043994, 0.67118841, 0.67526398,
       0.68162986, 0.67847923, 0.68673793, 0.63210551, 0.67178863,
       0.66847008, 0.67549638, 0.36632402, 0.65149325, 0.66340781,
       0.65171702, 0.64694625, 0.64290575, 0.3638055 , 0.71533396,
       0.67281088, 0.6529639 , 0.68983336, 0.69336802, 0.61210613,
       0.38898526, 0.68750039, 0.65872909, 0.70334778, 0.6748988 ,
       0.69878848, 0.63796471, 0.61812901, 0.66060563, 0.6828691 ,
       0.71371512, 0.65665613, 0.66584407, 0.6663671 , 0.3660821 ,
       0.63973084, 0.6838006 , 0.67559297, 0.65975556, 0.68347636,
       0.66793794, 0.66830675, 0.65510873, 0.67169949, 0.67760912,
       0.66402768, 0.61485843, 0.64511113, 0.65868458, 0.64952889,
       0.64604086, 0.62416117, 0.34418248, 0.34443406, 0.37572745,
       0.70218222, 0.6479473 , 0.39485303, 0.68547589, 0.66214134,
       0.679994  , 0.6536037 , 0.70521181, 0.70176157, 0.69609165,
       0.65537072, 0.62126577, 0.66750221, 0.37231359, 0.68788

In [16]:
#AUC ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob,pos_label='Y')
roc_auc = metrics.auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
# roc_auc = metrics.auc(fpr, tpr)
# print("Area under the ROC curve : %f" % roc_auc)

Area under the ROC curve : 0.771643


In [17]:
#Which Error is Costly??
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr,
index = i),'1-fpr' : pd.Series(1-fpr, index = i)})
print(roc)

         fpr       tpr     1-fpr
0   0.000000  0.000000  1.000000
1   0.000000  0.010526  1.000000
2   0.000000  0.073684  1.000000
3   0.020408  0.073684  0.979592
4   0.020408  0.305263  0.979592
5   0.040816  0.305263  0.959184
6   0.040816  0.326316  0.959184
7   0.061224  0.326316  0.938776
8   0.061224  0.347368  0.938776
9   0.081633  0.347368  0.918367
10  0.081633  0.368421  0.918367
11  0.102041  0.368421  0.897959
12  0.102041  0.421053  0.897959
13  0.142857  0.421053  0.857143
14  0.142857  0.442105  0.857143
15  0.183673  0.442105  0.816327
16  0.183673  0.484211  0.816327
17  0.204082  0.484211  0.795918
18  0.204082  0.589474  0.795918
19  0.224490  0.589474  0.775510
20  0.224490  0.600000  0.775510
21  0.265306  0.600000  0.734694
22  0.265306  0.631579  0.734694
23  0.285714  0.631579  0.714286
24  0.285714  0.642105  0.714286
25  0.326531  0.642105  0.673469
26  0.326531  0.652632  0.673469
27  0.346939  0.652632  0.653061
28  0.346939  0.663158  0.653061
29  0.3673

In [18]:
#Which Error is Costly??
#i = np.arange(len(tpr)) # index for df
roc_with_threshold = pd.DataFrame({'fpr' : pd.Series(fpr, index=thresholds),'tpr' : pd.Series(tpr,
index = thresholds),'1-fpr' : pd.Series(1-fpr, index = thresholds)})
print(roc_with_threshold)

               fpr       tpr     1-fpr
1.720237  0.000000  0.000000  1.000000
0.720237  0.000000  0.010526  1.000000
0.706825  0.000000  0.073684  1.000000
0.706150  0.020408  0.073684  0.979592
0.683801  0.020408  0.305263  0.979592
0.683476  0.040816  0.305263  0.959184
0.682869  0.040816  0.326316  0.959184
0.682484  0.061224  0.326316  0.938776
0.681643  0.061224  0.347368  0.938776
0.681630  0.081633  0.347368  0.918367
0.679994  0.081633  0.368421  0.918367
0.679413  0.102041  0.368421  0.897959
0.675593  0.102041  0.421053  0.897959
0.675264  0.142857  0.421053  0.857143
0.674637  0.142857  0.442105  0.857143
0.673660  0.183673  0.442105  0.816327
0.671699  0.183673  0.484211  0.816327
0.671188  0.204082  0.484211  0.795918
0.664028  0.204082  0.589474  0.795918
0.663904  0.224490  0.589474  0.775510
0.663463  0.224490  0.600000  0.775510
0.662141  0.265306  0.600000  0.734694
0.661124  0.265306  0.631579  0.734694
0.660606  0.285714  0.631579  0.714286
0.659756  0.285714  0.642